In [31]:
%%sh
pip install --use-pep517 -r requirements.txt

In [2]:
#TODO remove unused imports
#TODO wrap this whole script in a retraining.py file to be scheduled on a regular basis in e.g Glue or Lambda (depending how long it takes to run it)

import boto3
import sagemaker
import time
import random
import uuid
import logging
import stepfunctions
import io
import random
import os

from sagemaker.amazon.amazon_estimator import get_image_uri
from stepfunctions import steps
from stepfunctions.steps import TrainingStep, ModelStep, TransformStep
from stepfunctions.inputs import ExecutionInput
from stepfunctions.workflow import Workflow
from stepfunctions.template import TrainingPipeline
from stepfunctions.template.utils import replace_parameters_with_jsonpath


import os
from sagemaker import get_execution_role
from dotenv import load_dotenv
from load_data import load_data
from transform_data import split_data
import importlib
from save_model_to_s3 import save_model_to_s3
from deploy_model_endpoint import deploy_model
from finalize_and_save_model import finalize_and_save_model
from delete_sagemaker_endpoint import delete_sagemaker_endpoint

#TODO remove comment if not needed
# from ydata_profiling import ProfileReport
import boto3
import numpy as np

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
# Variables Setup Stage
load_dotenv(".env")
role = get_execution_role()

# Env variables
data_location_s3 = os.getenv("data_location_s3")
algorithm_choice = os.getenv("algorithm_choice")
target = os.getenv("target")
endpoint_name = os.getenv("endpoint_name")
model_name = os.getenv("model_name")
data_location = "s3://{}".format(data_location_s3)
instance_type = os.getenv("instance_type")
model_instance_count = int(os.getenv("model_instance_count"))
image_uri = os.getenv("ecr_repo_uri")
tuning_metric = os.getenv("tuning_metric")

#TODO change to log instead of print
print(
    data_location_s3,
    algorithm_choice,
    target,
    endpoint_name,
    model_name,
    data_location,
    instance_type,
    image_uri,
    tuning_metric,
    role
)

streaming-data-platform-ml-data/ethan_data.csv classification y classification-proba-endpoint banking-classification s3://streaming-data-platform-ml-data/ethan_data.csv ml.m4.xlarge 135544376709.dkr.ecr.eu-west-1.amazonaws.com/mlops-classification-repo:latest AUC arn:aws:iam::135544376709:role/banking-classification-sagemaker-role


In [4]:
if algorithm_choice.lower() == "classification": 
    objective = 'binary:logistic'
elif algorithm_choice.lower() == "regression":
    objective = 'reg:linear'

In [5]:
stepfunctions.set_stream_logger(level=logging.INFO)
bucket = "streaming-data-platform-ml-data" #TODO remove any hardcoded variables like this and pass them from Terraform
prefix = "step_function"
bucket_path = 's3://{}/{}/'.format(bucket, prefix)
region = boto3.Session().region_name
session = sagemaker.Session()

In [6]:
FILE_DATA = load_data(data_location)
print(FILE_DATA.head())

   age  job  education  default  balance  housing  loan  y
0   32    7          2        1     -238        1     0  0
1   34    4          2        0     -478        1     1  0
2   32    3          2        0      266        1     0  0
3   36    7          2        1       13        0     1  0
4   23   11          2        0      486        0     0  0


In [7]:
print(len(FILE_DATA)) #TODO add a more meaningful log message

55818


In [8]:
#TODO remove no need for this
FILE_DATA = FILE_DATA.sample(frac=1)
FILE_DATA.head()

,age,job,education,default,balance,housing,loan,y
41443,32,2,3,0,311,0,0,1
9958,40,2,3,0,270,0,0,0
20118,31,5,2,0,0,1,0,0
27354,29,7,1,0,201,1,0,0
39856,51,3,2,0,153,0,0,1


In [9]:
from sklearn.model_selection import train_test_split #TODO include this in other imports on top

train_and_val_data, test_data = split_data(FILE_DATA, shuffle=True)

In [10]:
train_data, validation_data = split_data(train_and_val_data, shuffle=True)

In [11]:
from sklearn.datasets import dump_svmlight_file
PERCENT_TRAIN = 70
PERCENT_VALIDATION = 15
X = FILE_DATA.drop(columns=['y'])  # Assuming 'y' is the name of the target column
y = FILE_DATA['y']
train_index = int(PERCENT_TRAIN / 100 * len(FILE_DATA))
validation_index = int((PERCENT_TRAIN + PERCENT_VALIDATION) / 100 * len(FILE_DATA))
train_data_x, validation_data_x, test_data_x = np.split(X, [train_index, validation_index])
train_data_y, validation_data_y, test_data_y = np.split(y, [train_index, validation_index])

In [12]:
print(train_data_x, train_data_y) 

       age  job  education  default  balance  housing  loan
41443   32    2          3        0      311        0     0
9958    40    2          3        0      270        0     0
20118   31    5          2        0        0        1     0
27354   29    7          1        0      201        1     0
39856   51    3          2        0      153        0     0
...    ...  ...        ...      ...      ...      ...   ...
11357   29    5          2        0     -205        0     0
40690   21    9          3        0      830        0     0
22558   65    8          2        0     1176        0     0
42179   36    2          3        0       62        0     0
4048    48    7          2        0      387        1     0

[39072 rows x 7 columns] 41443    1
9958     0
20118    0
27354    0
39856    1
        ..
11357    0
40690    1
22558    0
42179    1
4048     0
Name: y, Length: 39072, dtype: int64


In [13]:
FILE_TRAIN = "train.csv"
FILE_TEST = "test.csv"
FILE_VALIDATION = "validation.csv"

In [14]:
dump_svmlight_file(train_data_x, train_data_y, FILE_TRAIN)
dump_svmlight_file(validation_data_x, validation_data_y, FILE_VALIDATION)
dump_svmlight_file(test_data_x, test_data_y, FILE_TEST)

In [15]:
train_s3_file = os.path.join(prefix, 'train', FILE_TRAIN)
validation_s3_file = os.path.join(prefix, 'train', FILE_VALIDATION)
test_s3_file = os.path.join(prefix, 'train', FILE_TEST)

In [16]:
s3_client = boto3.client('s3')
s3_client.upload_file(FILE_TRAIN, bucket, train_s3_file)
s3_client.upload_file(FILE_VALIDATION, bucket, validation_s3_file)
s3_client.upload_file(FILE_TEST, bucket, test_s3_file)
train_s3_file = 's3://{}/{}'.format(bucket, train_s3_file)
validation_s3_file = 's3://{}/{}'.format(bucket, validation_s3_file)
test_s3_file = 's3://{}/{}'.format(bucket, test_s3_file)
output_s3  = 's3://{}/{}/{}/'.format(bucket, prefix, 'output')

In [17]:
sagemaker_execution_role = role
xgb = sagemaker.estimator.Estimator(
    get_image_uri(region, 'xgboost', repo_version='0.90-2'), #TODO pass the image name where we define other variables at the top
    sagemaker_execution_role, 
    train_instance_count = 1, 
    train_instance_type = 'ml.m4.4xlarge', #TODO this should be the same training instance that we pass from Terraform
    output_path = output_s3,
    sagemaker_session = session
)

xgb.set_hyperparameters(
    objective = objective, #TODO the objective should be the same as algorithm type from Terraform. Also we should do some basic hyperparameter tuning
    num_round = 50,
    max_depth = 5,
    eta = 0.2,
    gamma = 4,
    min_child_weight = 6,
    subsample = 0.7,
    silent = 0
)

In [18]:
execution_input = ExecutionInput(schema={
    'JobName': str, 
    'ModelName': str,
    'EndpointName': str
})

In [19]:
train_s3_file

's3://streaming-data-platform-ml-data/step_function/train/train.csv'

In [20]:
training_step = steps.TrainingStep(
    'Train Step', 
    estimator=xgb,
    data={
        'train': sagemaker.inputs.TrainingInput(train_s3_file, content_type='libsvm'),
        'validation': sagemaker.inputs.TrainingInput(validation_s3_file, content_type='libsvm')
    },
    job_name=execution_input['JobName']
)

In [21]:
model_step = steps.ModelStep(
    'Save model',
    model=training_step.get_expected_model(),
    model_name=execution_input['ModelName']  
)

In [22]:
#TODO ideally this should perform the same transformations as here https://github.com/crederauk/terraform-aws-mlops-module/blob/e3aa13fdbc02ce328b45fa466c07e3acc9e8632a/mlops_ml_models/transform_data.py#L39
# transform_step = steps.TransformStep(
#     'Transform Input Dataset',
#     transformer=xgb.transformer(
#         instance_count=1,
#         instance_type='ml.m5.large'
#     ),
#     job_name=execution_input['JobName'],     
#     model_name=execution_input['ModelName'], 
#     data=test_s3_file,
#     content_type='text/csv'
# )

In [23]:
endpoint_config_step = steps.EndpointConfigStep(
    "Create Endpoint Config",
    endpoint_config_name=execution_input['ModelName'],
    model_name=execution_input['ModelName'],
    initial_instance_count=1,
    instance_type='ml.m5.large' #TODO this should be the same endpoint instance type that we pass from Terraform
)

In [24]:
endpoint_step = steps.EndpointStep(
    "Create Endpoint",
    endpoint_name=execution_input['EndpointName'],
    endpoint_config_name=execution_input['ModelName']
)

In [25]:
workflow_definition = steps.Chain([
    training_step,
    model_step,
#     transform_step, #TODO bring back once we have the transform step working
    endpoint_config_step,
    endpoint_step
])

In [26]:
from time import strftime, gmtime #TODO import on the top as the other libraries
timestamp = strftime('%d-%H-%M-%S', gmtime()) #TODO define together with other variables at the top

workflow = Workflow(
    name='{}-{}'.format('MyTrainTransformDeploy_v1', timestamp), #TODO the name should be model_name variable from terraform with some version number
    definition=workflow_definition,
    role=role,
    execution_input=execution_input
)

In [27]:
workflow.render_graph()

In [28]:
workflow.create()

[INFO] Workflow created successfully on AWS Step Functions.


'arn:aws:states:eu-west-1:135544376709:stateMachine:MyTrainTransformDeploy_v1-04-18-03-58'

In [29]:
execution = workflow.execute(
    inputs={
        #TODO those names should come from Terraform
        'JobName': 'regression-{}'.format(uuid.uuid1().hex), # Each Sagemaker Job requires a unique name
        'ModelName': 'regression-{}'.format(uuid.uuid1().hex), # Each Model requires a unique name,
        'EndpointName': 'regression-{}'.format(uuid.uuid1().hex) # Each Endpoint requires a unique name,
    }
)

[INFO] Workflow execution started successfully on AWS Step Functions.


In [30]:
execution.render_progress()